In [1]:

### File Address: https://www.sample-videos.com/
%load_ext autoreload
%autoreload 2
import json
import time
import os
from blake3 import blake3
import warnings
warnings.filterwarnings("ignore")
video_path = "demo-3s-small.mp4"
input_height = 240
input_width = 320
output_height = 120
output_width = 160
current_dir = os.getcwd()
base_dir = os.path.dirname(current_dir)
snark_path = "/Users/ultraspicy/Developer/fibonacci/video-resizing-freivalds"
target_pk_file = os.path.join(base_dir, "demo", "test_pk.bin")
target_vk_file = os.path.join(base_dir, "demo", "test_vk.bin")


# Init ELF

In [2]:
start_time = time.time()
os.system(f"cd {snark_path}/program-precompile && RUSTFLAGS='-Awarnings' cargo prove build")
end_time = time.time()
print(f"Time taken to build ELF: {end_time - start_time} seconds")

cargo:warning=rustc +succinct --version: "rustc 1.85.0-dev\n"


[sp1]     Compiling proc-macro2 v1.0.93
[sp1]     Compiling unicode-ident v1.0.13
[sp1]     Compiling autocfg v1.3.0
[sp1]     Compiling libm v0.2.8
[sp1]     Compiling cfg-if v1.0.0
[sp1]     Compiling version_check v0.9.5
[sp1]     Compiling serde v1.0.210
[sp1]     Compiling byteorder v1.5.0
[sp1]     Compiling either v1.13.0
[sp1]     Compiling typenum v1.17.0
[sp1]     Compiling crunchy v0.2.2
[sp1]     Compiling once_cell v1.19.0
[sp1]     Compiling pin-project-lite v0.2.14
[sp1]     Compiling tiny-keccak v2.0.2
[sp1]     Compiling getrandom v0.2.15
[sp1]     Compiling paste v1.0.15
[sp1]     Compiling rand_core v0.6.4
[sp1]     Compiling itertools v0.12.1
[sp1]     Compiling tracing-core v0.1.32
[sp1]     Compiling p3-maybe-rayon v0.2.0-succinct
[sp1]     Compiling generic-array v0.14.7
[sp1]     Compiling num-traits v0.2.19
[sp1]     Compiling crossbeam-utils v0.8.20
[sp1]     Compiling syn v1.0.109
[sp1]     Compiling proc-macro-error-attr v1.0.4
[sp1]     Compiling proc-macro

cargo:rustc-env=SP1_ELF_freivalds-precompile=/Users/ultraspicy/Developer/fibonacci/video-resizing-freivalds/target/elf-compilation/riscv32im-succinct-zkvm-elf/release/freivalds-precompile
Time taken to build ELF: 9.320873975753784 seconds


[sp1]  warning: `freivalds-precompile` (bin "freivalds-precompile") generated 1 warning (run `cargo fix --bin "freivalds-precompile"` to apply 1 suggestion)
[sp1]      Finished `release` profile [optimized] target(s) in 8.59s


# Verifier Setup Pk and Vk

In [3]:
start_time = time.time()
config = {
    "target_pk_file": target_pk_file,
    "target_vk_file": target_vk_file
}
with open(os.path.join(snark_path, "init", "config.json"), "w") as f:
    json.dump(config, f)
os.system(f"cd {snark_path}/init && RUST_LOG=info cargo run --release -- --prove")
end_time = time.time()
print(f"Time taken to generate Pk and Vk: {end_time - start_time} seconds")

 --> init/src/main.rs:1:36
  |
1 | use sp1_sdk::{utils, ProverClient, SP1ProofWithPublicValues, SP1Stdin, SP1VerifyingKey};
  |                                    ^^^^^^^^^^^^^^^^^^^^^^^^  ^^^^^^^^  ^^^^^^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> init/src/main.rs:2:5
  |
2 | use std::io::Write;
  |     ^^^^^^^^^^^^^^

 --> init/src/main.rs:4:5
  |
4 | use clap::Parser;
  |     ^^^^^^^^^^^^

  --> init/src/main.rs:13:9
   |
13 |     let client = ProverClient::from_env();
   |         ^^^^^^ help: if this is intentional, prefix it with an underscore: `_client`
   |
   = note: `#[warn(unused_variables)]` on by default

    Finished `release` profile [optimized] target(s) in 0.42s
     Running `/Users/ultraspicy/Developer/fibonacci/video-resizing-freivalds/target/release/init --prove`


2025-06-03T03:14:08.019313Z  WARN SP1_PROVER environment variable not set, defaulting to 'cpu'    
2025-06-03T03:14:08.497583Z  INFO vk verification: true
2025-06-03T03:14:10.394831Z  WARN SP1_PROVER environment variable not set, defaulting to 'cpu'    
2025-06-03T03:14:10.849206Z  INFO vk verification: true
Successfully Save Pk and Vk
Time taken to generate Pk and Vk: 6.421773910522461 seconds


# Prover

### Decompose images from the video

In [4]:
# Use ffmpeg to decompose
import os
# Create directories if they don't exist
if not os.path.exists("frames"):
    os.makedirs("frames")
if not os.path.exists("decomposed_frames"):
    os.makedirs("decomposed_frames")

os.system(f"ffmpeg -i {video_path} frames/output_%04d.png")

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

0

In [5]:
import sys
sys.path.append(os.path.abspath("../"))
from scripts.decompose_image import decompose_image 
for i in sorted(os.listdir("frames")):
    if i.endswith(".png"):
        r,g,b = decompose_image(os.path.join("frames", i), os.path.join("decomposed_frames", i[:-4]))

Saved R channel to decomposed_frames/output_0001_R.txt
Saved G channel to decomposed_frames/output_0001_G.txt
Saved B channel to decomposed_frames/output_0001_B.txt
Saved R channel to decomposed_frames/output_0002_R.txt
Saved G channel to decomposed_frames/output_0002_G.txt
Saved B channel to decomposed_frames/output_0002_B.txt
Saved R channel to decomposed_frames/output_0003_R.txt
Saved G channel to decomposed_frames/output_0003_G.txt
Saved B channel to decomposed_frames/output_0003_B.txt
Saved R channel to decomposed_frames/output_0004_R.txt
Saved G channel to decomposed_frames/output_0004_G.txt
Saved B channel to decomposed_frames/output_0004_B.txt
Saved R channel to decomposed_frames/output_0005_R.txt
Saved G channel to decomposed_frames/output_0005_G.txt
Saved B channel to decomposed_frames/output_0005_B.txt
Saved R channel to decomposed_frames/output_0006_R.txt
Saved G channel to decomposed_frames/output_0006_G.txt
Saved B channel to decomposed_frames/output_0006_B.txt
Saved R ch

In [6]:
### Generate the targeted File 
## Shared knowledge between prover and verifier
if os.path.exists("resized.mp4"):
    os.remove("resized.mp4")
os.system(f"ffmpeg -i {video_path} -vf scale=160:120 resized.mp4")
if not os.path.exists("resized_frames"):
    os.makedirs("resized_frames")
if not os.path.exists("decomposed_resized_frames"):
    os.makedirs("decomposed_resized_frames")

# Check if resized.mp4 exists and ask for overwrite permission

os.system(f"ffmpeg -i resized.mp4 resized_frames/output_%04d.png")
for i in sorted(os.listdir("frames")):
    if i.endswith(".png"):
        r,g,b = decompose_image(os.path.join("resized_frames", i), os.path.join("decomposed_resized_frames", i[:-4]))

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

Saved R channel to decomposed_resized_frames/output_0001_R.txt
Saved G channel to decomposed_resized_frames/output_0001_G.txt
Saved B channel to decomposed_resized_frames/output_0001_B.txt
Saved R channel to decomposed_resized_frames/output_0002_R.txt
Saved G channel to decomposed_resized_frames/output_0002_G.txt
Saved B channel to decomposed_resized_frames/output_0002_B.txt
Saved R channel to decomposed_resized_frames/output_0003_R.txt
Saved G channel to decomposed_resized_frames/output_0003_G.txt
Saved B channel to decomposed_resized_frames/output_0003_B.txt
Saved R channel to decomposed_resized_frames/output_0004_R.txt
Saved G channel to decomposed_resized_frames/output_0004_G.txt
Saved B channel to decomposed_resized_frames/output_0004_B.txt
Saved R channel to decomposed_resized_frames/output_0005_R.txt
Saved G channel to decomposed_resized_frames/output_0005_G.txt
Saved B channel to decomposed_resized_frames/output_0005_B.txt
Saved R channel to decomposed_resized_frames/output_000

In [7]:
# Generate Proof from rust 
if not os.path.exists("proof"):
    os.makedirs("proof")
current_dir = os.getcwd()
base_dir = os.path.dirname(current_dir)
test_counts = 5
for i in sorted(os.listdir("decomposed_frames")):
    start_time = time.time()
    print("generating proof for ", i)
    if i.endswith(".txt"):
        config = {
            "input_width": input_width,
            "input_height": input_height,
            "output_width": output_width,
            "output_height": output_height,
            "input_file": os.path.join(base_dir, "demo/decomposed_frames", i),
            "target_file": os.path.join(base_dir, "demo/decomposed_resized_frames", i),
            "target_prove_file": os.path.join(base_dir, "demo/proof", i[:-4] + "_proof-with-pis.bin"),
            "target_pk_file": target_pk_file
        }
        with open(os.path.join(snark_path, "script", "config.json"), "w") as f:
            json.dump(config, f)
        os.system(f"cd {snark_path}/script && RUST_LOG=info cargo run --release -- --prove")
    end_time = time.time()
    print(f"Time taken to generate proof for {i}: {end_time - start_time} seconds")
    test_counts -= 1
    if test_counts == 0:
        break    


generating proof for  output_0001_B.txt


 --> script/src/bin/main.rs:1:11
  |
1 | use lib::{generate_horizontal_filter, generate_vertical_filter, load_image_from_file};
  |           ^^^^^^^^^^^^^^^^^^^^^^^^^^  ^^^^^^^^^^^^^^^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> script/src/bin/main.rs:2:72
  |
2 | use sp1_sdk::{utils, ProverClient, SP1ProofWithPublicValues, SP1Stdin, SP1VerifyingKey};
  |                                                                        ^^^^^^^^^^^^^^^

 --> script/src/bin/main.rs:5:5
  |
5 | use clap::Parser;
  |     ^^^^^^^^^^^^

 --> script/src/bin/main.rs:7:14
  |
7 | use blake3::{hash};
  |              ^^^^

   --> script/src/bin/main.rs:118:27
    |
118 |             let product = (freivalds_left[i] as u32 * h_matrix[i][j] as u32);
    |                           ^                                                ^
    |
    = note: `#[warn(unused_parens)]` on by default
help: remove these parentheses
    |
118 -             let product = (freivalds_left[i] as u32 * 

input_path: "/Users/ultraspicy/Developer/fibonacci/demo/decomposed_frames/output_0001_B.txt"
input_path: "/Users/ultraspicy/Developer/fibonacci/demo/decomposed_resized_frames/output_0001_B.txt"
2025-06-03T03:14:35.976131Z  WARN SP1_PROVER environment variable not set, defaulting to 'cpu'    
2025-06-03T03:14:36.461814Z  INFO vk verification: true
2025-06-03T03:14:38.636403Z  INFO prove_core: clk = 0 pc = 0x20209c    
2025-06-03T03:14:38.636551Z  INFO prove_core: ┌╴setup
2025-06-03T03:14:38.652672Z  INFO prove_core: clk = 0 pc = 0x20209c    
2025-06-03T03:14:38.652749Z  INFO prove_core: ┌╴setup
2025-06-03T03:14:38.668151Z  INFO prove_core: │ ┌╴commit
2025-06-03T03:14:38.668170Z  INFO prove_core: │ └╴380 cycles
2025-06-03T03:14:38.708213Z  INFO prove_core: deferred 0 records    
2025-06-03T03:14:38.755617Z  INFO prove_core: Shard Lifted: Index=1, Cluster=59
2025-06-03T03:14:38.755627Z  INFO prove_core: Chip Cpu: 20  -> 20 
2025-06-03T03:14:38.755628Z  INFO prove_core: Chip AddSub: 20  ->

stderr: cycle-tracker-start: main
stderr: cycle-tracker-start: main
stderr: cycle-tracker-end: main


2025-06-03T03:14:38.968178Z  INFO prove_core:generate main traces: close time.busy=212ms time.idle=750ns index=0
2025-06-03T03:14:39.037930Z  INFO prove_core: deferred 0 records    
2025-06-03T03:14:39.121382Z  INFO prove_core: Shard Lifted: Index=2, Cluster=59
2025-06-03T03:14:39.121392Z  INFO prove_core: Chip Cpu: 20  -> 20 
2025-06-03T03:14:39.121394Z  INFO prove_core: Chip AddSub: 20  -> 20 
2025-06-03T03:14:39.121396Z  INFO prove_core: Chip Bitwise: 17  -> 17 
2025-06-03T03:14:39.121397Z  INFO prove_core: Chip Mul: 16  -> 16 
2025-06-03T03:14:39.121397Z  INFO prove_core: Chip ShiftRight: 16  -> 16 
2025-06-03T03:14:39.121403Z  INFO prove_core: Chip ShiftLeft: 16  -> 16 
2025-06-03T03:14:39.121404Z  INFO prove_core: Chip Lt: 19  -> 19 
2025-06-03T03:14:39.121405Z  INFO prove_core: Chip MemoryLocal: 14  -> 17 
2025-06-03T03:14:39.121406Z  INFO prove_core: Chip MemoryInstrs: 19  -> 19 
2025-06-03T03:14:39.121407Z  INFO prove_core: Chip Auipc: 8   -> 17 
2025-06-03T03:14:39.121408Z  I

...

input_path: "/Users/ultraspicy/Developer/fibonacci/demo/decomposed_frames/output_0002_G.txt"
input_path: "/Users/ultraspicy/Developer/fibonacci/demo/decomposed_resized_frames/output_0002_G.txt"
2025-06-03T03:17:11.428178Z  WARN SP1_PROVER environment variable not set, defaulting to 'cpu'    
2025-06-03T03:17:11.920135Z  INFO vk verification: true
2025-06-03T03:17:14.158883Z  INFO prove_core: clk = 0 pc = 0x20209c    
2025-06-03T03:17:14.158996Z  INFO prove_core: ┌╴setup
2025-06-03T03:17:14.175470Z  INFO prove_core: clk = 0 pc = 0x20209c    
2025-06-03T03:17:14.175534Z  INFO prove_core: ┌╴setup
2025-06-03T03:17:14.192367Z  INFO prove_core: │ ┌╴commit
2025-06-03T03:17:14.192385Z  INFO prove_core: │ └╴380 cycles
2025-06-03T03:17:14.228165Z  INFO prove_core: deferred 0 records    
2025-06-03T03:17:14.273264Z  INFO prove_core: Shard Lifted: Index=1, Cluster=59
2025-06-03T03:17:14.273274Z  INFO prove_core: Chip Cpu: 20  -> 20 
2025-06-03T03:17:14.273276Z  INFO prove_core: Chip AddSub: 20  ->

stderr: cycle-tracker-start: main
stderr: cycle-tracker-start: main
stderr: cycle-tracker-end: main


2025-06-03T03:17:14.466708Z  INFO prove_core:generate main traces: close time.busy=193ms time.idle=1.00µs index=0
2025-06-03T03:17:14.526161Z  INFO prove_core: deferred 0 records    
2025-06-03T03:17:14.607226Z  INFO prove_core: Shard Lifted: Index=2, Cluster=59
2025-06-03T03:17:14.607236Z  INFO prove_core: Chip Cpu: 20  -> 20 
2025-06-03T03:17:14.607238Z  INFO prove_core: Chip AddSub: 20  -> 20 
2025-06-03T03:17:14.607239Z  INFO prove_core: Chip Bitwise: 17  -> 17 
2025-06-03T03:17:14.607240Z  INFO prove_core: Chip Mul: 16  -> 16 
2025-06-03T03:17:14.607241Z  INFO prove_core: Chip ShiftRight: 16  -> 16 
2025-06-03T03:17:14.607242Z  INFO prove_core: Chip ShiftLeft: 16  -> 16 
2025-06-03T03:17:14.607247Z  INFO prove_core: Chip Lt: 19  -> 19 
2025-06-03T03:17:14.607248Z  INFO prove_core: Chip MemoryLocal: 14  -> 17 
2025-06-03T03:17:14.607248Z  INFO prove_core: Chip MemoryInstrs: 19  -> 19 
2025-06-03T03:17:14.607249Z  INFO prove_core: Chip Auipc: 8   -> 17 
2025-06-03T03:17:14.607250Z  

stderr: cycle-tracker-end: main


2025-06-03T03:17:15.150510Z  INFO prove_core:generate main traces: close time.busy=312ms time.idle=1.12µs index=2
2025-06-03T03:17:46.940634Z  INFO prove_core: execution report (totals): total_cycles=2346541, total_syscall_cycles=71, touched_memory_addresses=87631
2025-06-03T03:17:46.940645Z  INFO prove_core: execution report (opcode counts):
2025-06-03T03:17:46.940650Z  INFO prove_core:   765058 add
2025-06-03T03:17:46.940652Z  INFO prove_core:   295504 lw
2025-06-03T03:17:46.940653Z  INFO prove_core:   197146 lbu
2025-06-03T03:17:46.940654Z  INFO prove_core:   193067 sw
2025-06-03T03:17:46.940656Z  INFO prove_core:   147713 sll
2025-06-03T03:17:46.940656Z  INFO prove_core:   147492 or
2025-06-03T03:17:46.940657Z  INFO prove_core:   116933 bne
2025-06-03T03:17:46.940658Z  INFO prove_core:   115577 bgeu
2025-06-03T03:17:46.940659Z  INFO prove_core:    96393 mul
2025-06-03T03:17:46.940660Z  INFO prove_core:    92162 srl
2025-06-03T03:17:46.940661Z  INFO prove_core:    74533 xor
2025-06-

# Verifier

In [8]:
if not os.path.exists("resized_frames"):
    os.makedirs("resized_frames")
if not os.path.exists("decomposed_resized_frames"):
    os.makedirs("decomposed_resized_frames")

# Check if resized.mp4 exists and ask for overwrite permission

os.system(f"ffmpeg -i resized.mp4 resized_frames/output_%04d.png")
for i in sorted(os.listdir("frames")):
    if i.endswith(".png"):
        r,g,b = decompose_image(os.path.join("resized_frames", i), os.path.join("decomposed_resized_frames", i[:-4]))

Saved R channel to decomposed_resized_frames/output_0001_R.txt
Saved G channel to decomposed_resized_frames/output_0001_G.txt
Saved B channel to decomposed_resized_frames/output_0001_B.txt
Saved R channel to decomposed_resized_frames/output_0002_R.txt
Saved G channel to decomposed_resized_frames/output_0002_G.txt
Saved B channel to decomposed_resized_frames/output_0002_B.txt
Saved R channel to decomposed_resized_frames/output_0003_R.txt
Saved G channel to decomposed_resized_frames/output_0003_G.txt
Saved B channel to decomposed_resized_frames/output_0003_B.txt
Saved R channel to decomposed_resized_frames/output_0004_R.txt
Saved G channel to decomposed_resized_frames/output_0004_G.txt
Saved B channel to decomposed_resized_frames/output_0004_B.txt
Saved R channel to decomposed_resized_frames/output_0005_R.txt
Saved G channel to decomposed_resized_frames/output_0005_G.txt
Saved B channel to decomposed_resized_frames/output_0005_B.txt
Saved R channel to decomposed_resized_frames/output_000

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

In [9]:
test_counts = 3
for i in sorted(os.listdir("decomposed_resized_frames")):
    print("verifying proof for ", i)
    if i.endswith(".txt"):
        start_time = time.time()
        prove_file = os.path.join(base_dir, "demo/proof", i[:-4] + "_proof-with-pis.bin")
        config = {
            "target_prove_file": prove_file,
            "target_vk_file": target_vk_file
        }
        with open(os.path.join(snark_path, "verifier", "config.json"), "w") as f:
            json.dump(config, f)
        os.system(f"cd {snark_path}/verifier && cargo run")
        data = []
        with open(os.path.join(base_dir, "demo/decomposed_resized_frames", i), "r") as f:
            for line in f:
                # Split line by spaces and convert each number
                numbers = [int(x) for x in line.strip().split()]
                data.extend(numbers)
        hash_obj = blake3(bytes(data))
        print(hash_obj.hexdigest())
        end_time = time.time()
        print(f"Time taken to verify proof for {i}: {end_time - start_time} seconds")
    test_counts -= 1
    if test_counts == 0:
        break

verifying proof for  output_0001_B.txt


 --> verifier/src/main.rs:1:62
  |
1 | use sp1_sdk::{utils, ProverClient, SP1ProofWithPublicValues, SP1Stdin, SP1VerifyingKey};
  |                                                              ^^^^^^^^  ^^^^^^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> verifier/src/main.rs:2:5
  |
2 | use std::io::Write;
  |     ^^^^^^^^^^^^^^

 --> verifier/src/main.rs:3:5
  |
3 | use blake3::hash;
  |     ^^^^^^^^^^^^

    Finished `dev` profile [unoptimized + debuginfo] target(s) in 0.56s
     Running `/Users/ultraspicy/Developer/fibonacci/video-resizing-freivalds/target/debug/verifier`


equal_sum: true
exceed_limit_20: 613
exceed_limit_50: 49
hash_target_image: Hash("c46149be2e66bc0b4899d725e14d3281df75453e961daf6c64fcf19911864990")
c46149be2e66bc0b4899d725e14d3281df75453e961daf6c64fcf19911864990
Time taken to verify proof for output_0001_B.txt: 21.864792823791504 seconds
verifying proof for  output_0001_G.txt


 --> verifier/src/main.rs:1:62
  |
1 | use sp1_sdk::{utils, ProverClient, SP1ProofWithPublicValues, SP1Stdin, SP1VerifyingKey};
  |                                                              ^^^^^^^^  ^^^^^^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> verifier/src/main.rs:2:5
  |
2 | use std::io::Write;
  |     ^^^^^^^^^^^^^^

 --> verifier/src/main.rs:3:5
  |
3 | use blake3::hash;
  |     ^^^^^^^^^^^^

    Finished `dev` profile [unoptimized + debuginfo] target(s) in 0.16s
     Running `/Users/ultraspicy/Developer/fibonacci/video-resizing-freivalds/target/debug/verifier`


equal_sum: true
exceed_limit_20: 121
exceed_limit_50: 0
hash_target_image: Hash("236150d752ae55abe27b9f8407379836f19e2e02cc9478efad79201ac140ffc5")
236150d752ae55abe27b9f8407379836f19e2e02cc9478efad79201ac140ffc5
Time taken to verify proof for output_0001_G.txt: 21.589430570602417 seconds
verifying proof for  output_0001_R.txt


 --> verifier/src/main.rs:1:62
  |
1 | use sp1_sdk::{utils, ProverClient, SP1ProofWithPublicValues, SP1Stdin, SP1VerifyingKey};
  |                                                              ^^^^^^^^  ^^^^^^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> verifier/src/main.rs:2:5
  |
2 | use std::io::Write;
  |     ^^^^^^^^^^^^^^

 --> verifier/src/main.rs:3:5
  |
3 | use blake3::hash;
  |     ^^^^^^^^^^^^

    Finished `dev` profile [unoptimized + debuginfo] target(s) in 0.15s
     Running `/Users/ultraspicy/Developer/fibonacci/video-resizing-freivalds/target/debug/verifier`


equal_sum: true
exceed_limit_20: 157
exceed_limit_50: 0
hash_target_image: Hash("3129a6601880af99970a4ac518d4ed7f2ee9ae609be67f1e6a67a044be0cd358")
3129a6601880af99970a4ac518d4ed7f2ee9ae609be67f1e6a67a044be0cd358
Time taken to verify proof for output_0001_R.txt: 21.964648962020874 seconds
